In [1]:
def schedule(DAYS):
    import numpy as np
    d = [    
      [3180, 600],
[3180, 600],
[1200, 15600],
[1200, 15600],
[6288, 18600],
[6288, 18600]



]  #  第幾分鐘delay，製造時間
    
 


    # ==== 參數設定(與演算法相關) ====
    BEFORE = 600



    NUM_ITERATION = 20			# 世代數(迴圈數)

    NUM_CHROME = 20				# 染色體個數
    NUM_BIT = 6		            # 訂單數

    Pc = 0.5    					# 交配率 (代表共執行Pc*NUM_CHROME/2次交配)
    Pm = 0.01   					# 突變率 (代表共要執行Pm*NUM_CHROME*NUM_BIT次突變)

    NUM_PARENT = NUM_CHROME                         # 父母的個數
    NUM_CROSSOVER = int(Pc * NUM_CHROME / 2)        # 交配的次數
    NUM_CROSSOVER_2 = NUM_CROSSOVER*2               # 上數的兩倍
    NUM_MUTATION = int(Pm * NUM_CHROME * NUM_BIT)   # 突變的次數

    np.random.seed(0)          # 若要每次跑得都不一樣的結果，就把這行註解掉


    # ==== 確保前後三角後三角順序====
    serial = []
    for i in range(NUM_BIT):
        if(i != NUM_BIT-1):
            if(d[i][0] == d[i+1][0]):
                serial.append(i)

        

    # ==== 基因演算法會用到的函式 ====
    def initPop():             # 初始化群體 (new)
        p = []
        
        for i in range(NUM_CHROME) :
            p.append(np.random.permutation(range(0, NUM_BIT)))
             # 產生 1, ..., NUM_BIT 的排列   
        return p

    def fitFunc(x):            # 適應度函數 (new)
        lateness = 0
        finish = 0
        finish = finish - BEFORE*DAYS               
        for i in range(NUM_BIT) :
            
            finish = finish + d[x[i]][0]
            if(finish > d[x[i]][1]):
                lateness = lateness + (finish - d[x[i]][1])
        return -lateness                    # 因為是最小化問題

    def evaluatePop(p):        # 評估群體之適應度
        return [fitFunc(p[i]) for i in range(len(p))]

    def selection(p, p_fit):   # 用二元競爭式選擇法來挑父母
        a = []

        for i in range(NUM_PARENT):
            [j, k] = np.random.choice(NUM_CHROME, 2, replace=False)  # 任選兩個index
            if p_fit[j] > p_fit[k] :                      # 擇優
                a.append(p[j].copy())
            else:
                a.append(p[k].copy())

        return a

    def crossover_uniform(p):           # 用均勻交配來繁衍子代 (new)
        a = []

        for i in range(NUM_CROSSOVER) :
            mask = np.random.randint(2, size=NUM_BIT)

            [j, k] = np.random.choice(NUM_PARENT, 2, replace=False)  # 任選兩個index
        
            child1, child2 = p[j].copy(), p[k].copy()
            remain1, remain2 = list(p[j].copy()), list(p[k].copy())     # 存還沒被用掉的城市
        
            for m in range(NUM_BIT):
                if mask[m] == 1 :
                    remain2.remove(child1[m])   # 砍掉 remain2 中的值是 child1[m]
                    remain1.remove(child2[m])   # 砍掉 remain1 中的值是 child2[m]
            
            t = 0
            for m in range(NUM_BIT):
                if mask[m] == 0 :
                    child1[m] = remain2[t]
                    child2[m] = remain1[t]
                    t += 1
            
            a.append(child1)
            a.append(child2)

        return a

    def mutation(p):	           # 突變 (new)
        for _ in range(NUM_MUTATION) :
            row = np.random.randint(NUM_CROSSOVER_2)  # 任選一個染色體
            [j, k] = np.random.choice(NUM_BIT, 2)  # 任選兩個基因
        
            p[row][j], p[row][k] = p[row][k], p[row][j]       # 此染色體的兩基因互換


    def sortChrome(a, a_fit):	    # a的根據a_fit由大排到小
        a_index = range(len(a))                         # 產生 0, 1, 2, ..., |a|-1 的 list
        
        a_fit, a_index = zip(*sorted(zip(a_fit,a_index), reverse=True)) # a_index 根據 a_fit 的大小由大到小連動的排序
    
        return [a[i] for i in a_index], a_fit           # 根據 a_index 的次序來回傳 a，並把對應的 fit 回傳

    def replace(p, p_fit, a, a_fit):            # 適者生存
        b = np.concatenate((p,a), axis=0)               # 把本代 p 和子代 a 合併成 b
        b_fit = p_fit + a_fit                           # 把上述兩代的 fitness 合併成 b_fit
        
        b, b_fit = sortChrome(b, b_fit)                 # b 和 b_fit 連動的排序
        
        return b[:NUM_CHROME], list(b_fit[:NUM_CHROME]) # 回傳 NUM_CHROME 個為新的一個世代


    # ==== 主程式 ====

    pop = initPop()             # 初始化 pop
    pop_fit = evaluatePop(pop)  # 算 pop 的 fit

    best_outputs = []                           # 用此變數來紀錄每一個迴圈的最佳解 (new)
    best_outputs.append(np.max(pop_fit))        # 存下初始群體的最佳解

    mean_outputs = []                           # 用此變數來紀錄每一個迴圈的平均解 (new)
    mean_outputs.append(np.average(pop_fit))        # 存下初始群體的最佳解

    for i in range(NUM_ITERATION) :
        parent = selection(pop, pop_fit)            # 挑父母
        offspring = crossover_uniform(parent)       # 均勻交配
        mutation(offspring)                         # 突變
        offspring_fit = evaluatePop(offspring)      # 算子代的 fit
        pop, pop_fit = replace(pop, pop_fit, offspring, offspring_fit)    # 取代

        best_outputs.append(np.max(pop_fit))        # 存下這次的最佳解
        mean_outputs.append(np.average(pop_fit))
        


    for i in range(len(serial)):
        if(np.where(pop[0] == serial[i])[0][0] > np.where(pop[0] == serial[i]+1)[0][0]):
            pop[0][np.where(pop[0] == serial[i])[0][0]] = serial[i]+1
            pop[0][np.where(pop[0] == serial[i]+1)[0][0]] = serial[i]
            
    print('Beforehand %d days: x = %s, y = %d'	%(DAYS, pop[0]+1, -pop_fit[0]))


In [2]:
for i in range(20):
    schedule(i)

Beforehand 0 days: x = [1 2 3 4 5 6], y = 11076
Beforehand 1 days: x = [1 2 3 4 5 6], y = 9276
Beforehand 2 days: x = [1 2 3 5 4 6], y = 7476
Beforehand 3 days: x = [1 2 3 5 4 6], y = 5676
Beforehand 4 days: x = [1 2 3 5 4 6], y = 3876
Beforehand 5 days: x = [1 2 3 5 4 6], y = 2760
Beforehand 6 days: x = [1 2 3 5 4 6], y = 2160
Beforehand 7 days: x = [1 2 3 4 5 6], y = 1560
Beforehand 8 days: x = [1 2 3 5 4 6], y = 960
Beforehand 9 days: x = [1 2 3 5 4 6], y = 360
Beforehand 10 days: x = [1 2 5 6 3 4], y = 0
Beforehand 11 days: x = [1 2 5 6 3 4], y = 0
Beforehand 12 days: x = [1 3 2 5 6 4], y = 0
Beforehand 13 days: x = [1 3 2 5 6 4], y = 0
Beforehand 14 days: x = [1 3 2 5 6 4], y = 0
Beforehand 15 days: x = [1 3 2 5 6 4], y = 0
Beforehand 16 days: x = [1 3 2 5 6 4], y = 0
Beforehand 17 days: x = [1 3 2 5 6 4], y = 0
Beforehand 18 days: x = [1 3 2 5 6 4], y = 0
Beforehand 19 days: x = [1 3 2 5 6 4], y = 0
